In [1]:
#######################################################
# Much of this comes from https://www.kaggle.com/pradeeppathak9/gamma-log-facies-type-prediction
# https://www.crowdanalytix.com/contests/gamma-log-facies-type-prediction
######################################################
import os
os.system('pip install pytorch_toolbelt')
import pandas as pd
import numpy as np
import json
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
import time

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
from sklearn.model_selection import KFold
import gc

from tqdm import tqdm
from itertools import groupby, accumulate
from random import shuffle

from sklearn.model_selection import GroupKFold, GroupShuffleSplit, LeaveOneGroupOut
from sklearn.preprocessing import MinMaxScaler
from pytorch_toolbelt import losses as L

In [2]:
ss = pd.read_csv("../input/sample_submission.csv", dtype={'time':str})
train = pd.read_csv('../input/train_clean.csv')
train['filter'] = 0
test = pd.read_csv('../input/test_clean.csv')
test['filter'] = 2
ts1 = pd.concat([train, test], axis=0, sort=False).reset_index(drop=True)

# ts1['time2'] = pd.cut(ts1['time'], bins=np.linspace(0.0000, 700., num=14 + 1), labels=list(range(14)), include_lowest=True).astype(int)
# ts1['time2'] = ts1.groupby('time2')['time'].rank( )/500000.
# (never used)

np.random.seed(321)
ts1['group'] = pd.cut(ts1['time'], bins=np.linspace(0.0000, 700., num=14*125 + 1), labels=list(range(14*125)), include_lowest=True).astype(int)
# --- there might be an improvement ---
np.random.seed(321)

y = ts1.loc[ts1['filter']==0, 'open_channels']
group = ts1.loc[ts1['filter']==0, 'group']
X = ts1.loc[ts1['filter']==0, 'signal']

In [3]:
ts1.head()

,time,signal,open_channels,filter,group
0,0.0001,-2.7600,0.0,0,0
1,0.0002,-2.8557,0.0,0,0
2,0.0003,-2.4074,0.0,0,0
3,0.0004,-3.1404,0.0,0,0
4,0.0005,-3.1525,0.0,0,0


In [4]:
ts1[ts1.group==0].shape

(4000, 5)

In [5]:
print(X.shape, y.shape)

(5000000,) (5000000,)


In [6]:
X.head()

0   -2.7600
1   -2.8557
2   -2.4074
3   -3.1404
4   -3.1525
Name: signal, dtype: float64

In [7]:
np.random.seed(321)
skf = GroupKFold(n_splits=3) # there might be a problem, I would go with K-folder
splits = [x for x in skf.split(X, y, group)]

use_cols = [col for col in ts1.columns if col not in ['index','filter','group', 'open_channels', 'time', 'time2']]  
print(use_cols)

['signal']


In [8]:
# # Create numpy array of inputs  # there is no NA vals
# for col in use_cols:
#     col_mean = ts1[col].mean()
#     ts1[col] = ts1[col].fillna(col_mean)

In [9]:
val_preds_all = np.zeros((ts1[ts1['filter']==0].shape[0], 11)) # why 11?
test_preds_all = np.zeros((ts1[ts1['filter']==2].shape[0], 11))

groups = ts1.loc[ts1['filter']==0, 'group']
times = ts1.loc[ts1['filter']==0, 'time']

In [10]:
new_splits = []
for sp in splits:
    new_split = []
    new_split.append(np.unique(groups[sp[0]]))
    new_split.append(np.unique(groups[sp[1]]))
    new_splits.append(new_split)

In [11]:
trainval = np.array(list(ts1[ts1['filter']==0].groupby('group').apply(lambda x: x[use_cols].values)))
test = np.array(list(ts1[ts1['filter']==2].groupby('group').apply(lambda x: x[use_cols].values)))
trainval_y = np.array(list(ts1[ts1['filter']==0].groupby('group').apply(lambda x: x[['open_channels']].values)))

gc.collect()

print(trainval.shape, test.shape, trainval_y.shape)

(1250, 4000, 1) (500, 4000, 1) (1250, 4000, 1)


In [12]:
# transpose to B x C x L
trainval = trainval.transpose((0,2,1))
test = test.transpose((0,2,1))

trainval_y = trainval_y.reshape(trainval_y.shape[:2])
test_y = np.zeros((test.shape[0], trainval_y.shape[1]))

print(trainval.shape, test.shape, trainval_y.shape, test_y.shape)

(1250, 1, 4000) (500, 1, 4000) (1250, 4000) (500, 4000)


In [13]:
trainval = torch.Tensor(trainval)
test = torch.Tensor(test)

print(trainval.shape, test.shape)

torch.Size([1250, 1, 4000]) torch.Size([500, 1, 4000])


In [14]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0, checkpoint_path='checkpoint.pt', is_maximize=True):
        self.patience, self.delta, self.checkpoint_path = patience, delta, checkpoint_path
        self.counter, self.best_score = 0, None
        self.is_maximize = is_maximize

    def load_best_weights(self, model):
        model.load_state_dict(torch.load(self.checkpoint_path))

    def __call__(self, score, model):
        if self.best_score is None or \
        (score > self.best_score + self.delta if self.is_maximize else score < self.best_score - self.delta):
            torch.save(model.state_dict(), self.checkpoint_path)
            self.best_score, self.counter = score, 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [15]:
class Seq2SeqRnn(nn.Module):
    def __init__(self, input_size, seq_len, hidden_size, output_size, num_layers=1, bidirectional=False, dropout=.3,
            hidden_layers = [100, 200]):
        
        super().__init__()
        self.input_size = input_size
        self.seq_len = seq_len
        self.hidden_size = hidden_size
        self.num_layers=num_layers
        self.bidirectional=bidirectional
        self.output_size=output_size
        
        self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, 
                           bidirectional=bidirectional, batch_first=True,dropout=0.3)
         # Input Layer
        if hidden_layers and len(hidden_layers):
            first_layer  = nn.Linear(hidden_size*2 if bidirectional else hidden_size, hidden_layers[0])

            # Hidden Layers
            self.hidden_layers = nn.ModuleList(
                [first_layer]+[nn.Linear(hidden_layers[i], hidden_layers[i+1]) for i in range(len(hidden_layers) - 1)]
            )
            for layer in self.hidden_layers: nn.init.kaiming_normal_(layer.weight.data)   

            self.intermediate_layer = nn.Linear(hidden_layers[-1], self.input_size)
            # output layers
            self.output_layer = nn.Linear(hidden_layers[-1], output_size)
            nn.init.kaiming_normal_(self.output_layer.weight.data) 
           
        else:
            self.hidden_layers = []
            self.intermediate_layer = nn.Linear(hidden_size*2 if bidirectional else hidden_siz, self.input_size)
            self.output_layer = nn.Linear(hidden_size*2 if bidirectional else hidden_size, output_size)
            nn.init.kaiming_normal_(self.output_layer.weight.data) 

        self.activation_fn = torch.relu
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        batch_size = x.size(0)
        x = x.permute(0,2,1)

        outputs, hidden = self.rnn(x)        

        x = self.dropout(self.activation_fn(outputs))
        for hidden_layer in self.hidden_layers:
            x = self.activation_fn(hidden_layer(x))
            x = self.dropout(x)
            
        x = self.output_layer(x)

        return x


In [16]:
class IonDataset(Dataset):
    """Ion dataset."""

    def __init__(self, data, labels, training=True, transform=None, flip=0.5, noise_level=0, class_split=0.0):
        self.data = data
        self.labels = labels
        self.transform = transform
        self.training = training
        self.flip = flip
        self.noise_level = noise_level
        self.class_split = class_split

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        data = self.data[idx]
        labels = self.labels[idx]
        if np.random.rand() < self.class_split:
            data, labels = class_split(data, labels)
        if  np.random.rand() < self.noise_level:
            data = data * torch.FloatTensor(10000).uniform_(1-self.noise_level, 1+self.noise_level)
        if np.random.rand() < self.flip:
            data = torch.flip(data, dims=[1])
            labels = np.flip(labels, axis=0).copy().astype(int)

        return [data, labels.astype(int)]


In [17]:
if not os.path.exists("./models"):
            os.makedirs("./models")
for index, (train_index, val_index ) in enumerate(new_splits[0:], start=0):
    print("Fold : {}".format(index))
    
    batchsize = 128
    train_dataset = IonDataset(trainval[train_index],  trainval_y[train_index], flip=False, noise_level=0.0, class_split=0.0)
    train_dataloader = DataLoader(train_dataset, batchsize, shuffle=True, num_workers=16, pin_memory=True)

    valid_dataset = IonDataset(trainval[val_index],  trainval_y[val_index], flip=False)
    valid_dataloader = DataLoader(valid_dataset, batchsize, shuffle=False, num_workers=16, pin_memory=True)

    test_dataset = IonDataset(test,  test_y, flip=False, noise_level=0.0, class_split=0.0)
    test_dataloader = DataLoader(test_dataset, batchsize, shuffle=False, num_workers=16, pin_memory=True)
    test_preds_iter = np.zeros((2000000, 11))

    for it in range(1):
        device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
        model=Seq2SeqRnn(input_size=trainval.shape[1], seq_len=4000, hidden_size=64, output_size=11, num_layers=2, hidden_layers=[64,64,64],
                         bidirectional=True).to(device)
        
        no_of_epochs = 150
        early_stopping = EarlyStopping(patience=20, is_maximize=True, checkpoint_path="./models/gru_clean_checkpoint_fold_{}_iter_{}.pt".format(index, it))
        criterion = L.FocalLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        schedular = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e2, max_lr=0.01, epochs=no_of_epochs,
                                                steps_per_epoch=len(train_dataloader))
        avg_train_losses, avg_valid_losses = [], [] 
    
    
        for epoch in range(no_of_epochs):
            start_time = time.time()
    
            print("Epoch : {}".format(epoch))
            print( "learning_rate: {:0.9f}".format(schedular.get_lr()[0]))
            train_losses, valid_losses = [], []
    
            model.train() # prep model for training
            train_preds, train_true = torch.Tensor([]).to(device), torch.LongTensor([]).to(device)
    
            for x, y in train_dataloader:
                x = x.to(device)
                y = y.to(device)
    
                optimizer.zero_grad()
                predictions = model(x[:, :trainval.shape[1], :])
    
                predictions_ = predictions.view(-1, predictions.shape[-1]) 
                y_ = y.view(-1)
    
                loss = criterion(predictions_, y_)
                # backward pass: compute gradient of the loss with respect to model parameters
                loss.backward()
                # perform a single optimization step (parameter update)
                optimizer.step()
                schedular.step()
                # record training lossa
                train_losses.append(loss.item())
    
                train_true = torch.cat([train_true, y_], 0)
                train_preds = torch.cat([train_preds, predictions_], 0)

            model.eval() # prep model for evaluation
            val_preds, val_true = torch.Tensor([]).to(device), torch.LongTensor([]).to(device)
            with torch.no_grad():
                for x, y in valid_dataloader:
                    x = x.to(device)
                    y = y.to(device)
    
                    predictions = model(x[:,:trainval.shape[1],:])
                    predictions_ = predictions.view(-1, predictions.shape[-1]) 
                    y_ = y.view(-1)
    
                    loss = criterion(predictions_, y_)
                    valid_losses.append(loss.item())
        
                    val_true = torch.cat([val_true, y_], 0)
                    val_preds = torch.cat([val_preds, predictions_], 0)

            # calculate average loss over an epoch
            train_loss = np.average(train_losses)
            valid_loss = np.average(valid_losses)
            avg_train_losses.append(train_loss)
            avg_valid_losses.append(valid_loss)
            
            print( "train_loss: {:0.6f}, valid_loss: {:0.6f}".format(train_loss, valid_loss))

            train_score = f1_score(train_true.cpu().detach().numpy(), train_preds.cpu().detach().numpy().argmax(1), labels=list(range(11)), average='macro')
            val_score = f1_score(val_true.cpu().detach().numpy(), val_preds.cpu().detach().numpy().argmax(1), labels=list(range(11)), average='macro')
            print( "train_f1: {:0.6f}, valid_f1: {:0.6f}".format(train_score, val_score))
    
            if early_stopping(val_score, model):
                print("Early Stopping...")
                print("Best Val Score: {:0.6f}".format(early_stopping.best_score))
                break
    
            print("--- %s seconds ---" % (time.time() - start_time))
        
        model.load_state_dict(torch.load("./models/gru_clean_checkpoint_fold_{}_iter_{}.pt".format(index, it)))
        with torch.no_grad():
            pred_list = []
            for x, y in test_dataloader:
                x = x.to(device)
                y = y.to(device)

                predictions = model(x[:,:trainval.shape[1],:])
                predictions_ = predictions.view(-1, predictions.shape[-1]) 

                pred_list.append(F.softmax(predictions_, dim=1).cpu().numpy())
            test_preds = np.vstack(pred_list)
       
        test_preds_iter += test_preds
        test_preds_all += test_preds
        if not os.path.exists("./predictions/test"):
            os.makedirs("./predictions/test")
        np.save('./predictions/test/gru_clean_fold_{}_iter_{}_raw.npy'.format(index, it), arr=test_preds_iter)
        np.save('./predictions/test/gru_clean_fold_{}_raw.npy'.format(index), arr=test_preds_all)

Fold : 0
Epoch : 0
learning_rate: 0.000100000
train_loss: 2.186580, valid_loss: 1.845660
train_f1: 0.045292, valid_f1: 0.018308
--- 7.415477514266968 seconds ---
Epoch : 1
learning_rate: 0.000210252
train_loss: 1.908828, valid_loss: 1.566396
train_f1: 0.054074, valid_f1: 0.025139
--- 7.729543209075928 seconds ---
Epoch : 2
learning_rate: 0.000536096
train_loss: 1.585143, valid_loss: 1.198098
train_f1: 0.075707, valid_f1: 0.094982
--- 7.414370536804199 seconds ---
Epoch : 3
learning_rate: 0.001063017
train_loss: 1.261296, valid_loss: 0.817176
train_f1: 0.099017, valid_f1: 0.088155
--- 7.714250087738037 seconds ---
Epoch : 4
learning_rate: 0.001767543
train_loss: 1.009829, valid_loss: 0.745155
train_f1: 0.129883, valid_f1: 0.075647
--- 7.32867169380188 seconds ---
Epoch : 5
learning_rate: 0.002618290
train_loss: 0.816270, valid_loss: 0.648013
train_f1: 0.161181, valid_f1: 0.198276
--- 8.332963705062866 seconds ---
Epoch : 6
learning_rate: 0.003577360
train_loss: 0.695454, valid_loss: 0.5

train_loss: 0.083382, valid_loss: 0.085191
train_f1: 0.844158, valid_f1: 0.909571
--- 7.440787076950073 seconds ---
Epoch : 54
learning_rate: 0.008065194
train_loss: 0.083273, valid_loss: 0.083980
train_f1: 0.849426, valid_f1: 0.858283
--- 7.554769039154053 seconds ---
Epoch : 55
learning_rate: 0.007972445
train_loss: 0.082544, valid_loss: 0.082920
train_f1: 0.852907, valid_f1: 0.846079
--- 7.7946226596832275 seconds ---
Epoch : 56
learning_rate: 0.007878087
train_loss: 0.081674, valid_loss: 0.082950
train_f1: 0.852757, valid_f1: 0.930233
--- 7.5721821784973145 seconds ---
Epoch : 57
learning_rate: 0.007782171
train_loss: 0.082213, valid_loss: 0.082118
train_f1: 0.859070, valid_f1: 0.916921
--- 7.842922210693359 seconds ---
Epoch : 58
learning_rate: 0.007684748
train_loss: 0.081123, valid_loss: 0.082010
train_f1: 0.860000, valid_f1: 0.900025
--- 7.745798587799072 seconds ---
Epoch : 59
learning_rate: 0.007585871
train_loss: 0.080693, valid_loss: 0.081838
train_f1: 0.863300, valid_f1: 0

train_f1: 0.898694, valid_f1: 0.935497
--- 7.442668676376343 seconds ---
Epoch : 107
learning_rate: 0.002287276
train_loss: 0.071798, valid_loss: 0.074703
train_f1: 0.897726, valid_f1: 0.935788
--- 7.184764862060547 seconds ---
Epoch : 108
learning_rate: 0.002190278
train_loss: 0.071933, valid_loss: 0.074950
train_f1: 0.898937, valid_f1: 0.935618
--- 7.582178115844727 seconds ---
Epoch : 109
learning_rate: 0.002094802
train_loss: 0.070248, valid_loss: 0.074758
train_f1: 0.899167, valid_f1: 0.935531
--- 7.312358379364014 seconds ---
Epoch : 110
learning_rate: 0.002000899
train_loss: 0.071310, valid_loss: 0.074666
train_f1: 0.899601, valid_f1: 0.935673
--- 7.632787227630615 seconds ---
Epoch : 111
learning_rate: 0.001908619
train_loss: 0.071380, valid_loss: 0.074793
train_f1: 0.899161, valid_f1: 0.935671
--- 7.253572940826416 seconds ---
Epoch : 112
learning_rate: 0.001818014
train_loss: 0.070880, valid_loss: 0.074504
train_f1: 0.899375, valid_f1: 0.935612
--- 7.3435704708099365 seconds 

train_f1: 0.717229, valid_f1: 0.837544
--- 7.267252206802368 seconds ---
Epoch : 20
learning_rate: 0.009964235
train_loss: 0.152288, valid_loss: 0.136460
train_f1: 0.732686, valid_f1: 0.804837
--- 7.333908557891846 seconds ---
Epoch : 21
learning_rate: 0.009949000
train_loss: 0.173239, valid_loss: 0.155341
train_f1: 0.700498, valid_f1: 0.831905
--- 7.453807353973389 seconds ---
Epoch : 22
learning_rate: 0.009931085
train_loss: 0.161333, valid_loss: 0.124429
train_f1: 0.722510, valid_f1: 0.811747
--- 7.56661319732666 seconds ---
Epoch : 23
learning_rate: 0.009910499
train_loss: 0.143337, valid_loss: 0.119229
train_f1: 0.741670, valid_f1: 0.814081
--- 7.738405704498291 seconds ---
Epoch : 24
learning_rate: 0.009887255
train_loss: 0.130614, valid_loss: 0.112373
train_f1: 0.759422, valid_f1: 0.831245
--- 7.240250825881958 seconds ---
Epoch : 25
learning_rate: 0.009861364
train_loss: 0.124436, valid_loss: 0.105702
train_f1: 0.768781, valid_f1: 0.840925
--- 7.422787666320801 seconds ---
Epoc

train_loss: 0.075889, valid_loss: 0.077276
train_f1: 0.880525, valid_f1: 0.935831
--- 7.6561126708984375 seconds ---
Epoch : 74
learning_rate: 0.005966284
train_loss: 0.076327, valid_loss: 0.077347
train_f1: 0.880745, valid_f1: 0.935598
--- 7.821583271026611 seconds ---
Epoch : 75
learning_rate: 0.005851871
train_loss: 0.077177, valid_loss: 0.077098
train_f1: 0.881271, valid_f1: 0.936369
--- 7.667788743972778 seconds ---
Epoch : 76
learning_rate: 0.005736996
train_loss: 0.076726, valid_loss: 0.077225
train_f1: 0.882042, valid_f1: 0.936354
--- 7.718796730041504 seconds ---
Epoch : 77
learning_rate: 0.005621723
train_loss: 0.076800, valid_loss: 0.076926
train_f1: 0.882694, valid_f1: 0.936126
--- 8.388787508010864 seconds ---
Epoch : 78
learning_rate: 0.005506113
train_loss: 0.077302, valid_loss: 0.076770
train_f1: 0.881747, valid_f1: 0.936310
--- 7.229533433914185 seconds ---
Epoch : 79
learning_rate: 0.005390229
train_loss: 0.076855, valid_loss: 0.077054
train_f1: 0.883014, valid_f1: 0.

train_f1: 0.890304, valid_f1: 0.937501
--- 8.06304407119751 seconds ---
Epoch : 127
learning_rate: 0.000690809
train_loss: 0.072314, valid_loss: 0.074967
train_f1: 0.890297, valid_f1: 0.937465
--- 8.031936168670654 seconds ---
Epoch : 128
learning_rate: 0.000632968
train_loss: 0.072831, valid_loss: 0.074943
train_f1: 0.890673, valid_f1: 0.937456
--- 7.555810213088989 seconds ---
Epoch : 129
learning_rate: 0.000577492
train_loss: 0.072314, valid_loss: 0.075029
train_f1: 0.890137, valid_f1: 0.937529
--- 7.449528217315674 seconds ---
Epoch : 130
learning_rate: 0.000524411
train_loss: 0.072901, valid_loss: 0.075020
train_f1: 0.891381, valid_f1: 0.937459
--- 7.446623086929321 seconds ---
Epoch : 131
learning_rate: 0.000473753
train_loss: 0.073317, valid_loss: 0.074996
train_f1: 0.890755, valid_f1: 0.937483
--- 7.499983787536621 seconds ---
Epoch : 132
learning_rate: 0.000425547
train_loss: 0.072906, valid_loss: 0.074985
train_f1: 0.890577, valid_f1: 0.937480
--- 7.313392877578735 seconds --

train_loss: 0.086663, valid_loss: 0.082111
train_f1: 0.853074, valid_f1: 0.847145
--- 7.262159585952759 seconds ---
Epoch : 41
learning_rate: 0.009102899
train_loss: 0.084007, valid_loss: 0.082207
train_f1: 0.854581, valid_f1: 0.844205
--- 7.2578349113464355 seconds ---
Epoch : 42
learning_rate: 0.009035293
train_loss: 0.085239, valid_loss: 0.080514
train_f1: 0.859700, valid_f1: 0.856836
--- 7.251209259033203 seconds ---
Epoch : 43
learning_rate: 0.008965503
train_loss: 0.085915, valid_loss: 0.083742
train_f1: 0.861927, valid_f1: 0.842855
--- 7.32746148109436 seconds ---
Epoch : 44
learning_rate: 0.008893564
train_loss: 0.083060, valid_loss: 0.078721
train_f1: 0.864516, valid_f1: 0.906312
--- 7.254142761230469 seconds ---
Epoch : 45
learning_rate: 0.008819518
train_loss: 0.081665, valid_loss: 0.081218
train_f1: 0.869630, valid_f1: 0.876332
--- 7.262022018432617 seconds ---
Epoch : 46
learning_rate: 0.008743403
train_loss: 0.082322, valid_loss: 0.077850
train_f1: 0.871456, valid_f1: 0.9

train_f1: 0.908350, valid_f1: 0.935779
--- 7.2628333568573 seconds ---
Epoch : 94
learning_rate: 0.003661812
train_loss: 0.069989, valid_loss: 0.070301
train_f1: 0.908834, valid_f1: 0.936300
--- 7.398880243301392 seconds ---
Epoch : 95
learning_rate: 0.003550074
train_loss: 0.067946, valid_loss: 0.070921
train_f1: 0.908894, valid_f1: 0.935368
--- 7.397958755493164 seconds ---
Epoch : 96
learning_rate: 0.003439122
train_loss: 0.069828, valid_loss: 0.070431
train_f1: 0.909078, valid_f1: 0.935767
--- 7.279065132141113 seconds ---
Epoch : 97
learning_rate: 0.003329014
train_loss: 0.067742, valid_loss: 0.070244
train_f1: 0.908979, valid_f1: 0.936201
--- 7.250555038452148 seconds ---
Epoch : 98
learning_rate: 0.003219811
train_loss: 0.068912, valid_loss: 0.070144
train_f1: 0.909395, valid_f1: 0.936552
--- 7.242939710617065 seconds ---
Epoch : 99
learning_rate: 0.003111572
train_loss: 0.069162, valid_loss: 0.070692
train_f1: 0.909683, valid_f1: 0.935542
--- 7.261530637741089 seconds ---
Epoch

In [20]:
# Best Val Score: 0.936947

test_preds_all = test_preds_all/np.sum(test_preds_all, axis=1)[:, None]
test_pred_frame = pd.DataFrame({'time': ss['time'].astype(str),
                                'open_channels': np.argmax(test_preds_all, axis=1)})
test_pred_frame.to_csv("../submissions/gru_preds.csv", index=False)

In [1]:
import os

In [16]:
LOG_DATE = "2020_04_14"
NOTE = "baseline"
FILE = "./logs/" + str(LOG_NAME) + "_" + str(NOTE) + ".log"
if not os.path.exists(FILE):
    os.mknod(FILE)